In [1]:
import ray

In [2]:
@ray.remote
def f():
    return 1


@ray.remote
def g():
    # Call f 4 times and return the resulting object refs.
    return [f.remote() for _ in range(4)]


@ray.remote
def h():
    # Call f 4 times, block until those 4 tasks finish,
    # retrieve the results, and return the values.
    return ray.get([f.remote() for _ in range(4)])

print(ray.get(g.remote()))
print(ray.get(h.remote()))

2024-12-13 18:43:06,410	INFO worker.py:1812 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8267 


[ObjectRef(9930f5acb950220cffffffffffffffffffffffff0100000001000000), ObjectRef(2a7e136b77f1e9fdffffffffffffffffffffffff0100000001000000), ObjectRef(e0b125015092bdf9ffffffffffffffffffffffff0100000001000000), ObjectRef(43476cf6705748a9ffffffffffffffffffffffff0100000001000000)]
[1, 1, 1, 1]


In [3]:
# Yielding Resources While Blocked
@ray.remote(num_cpus=1, num_gpus=1)
def g():
    return ray.get(f.remote())

# 在执行嵌套任务时，当遇到阻塞情况，则会释放cpu，当需要return时，重新获取cpu，这样可以避免死锁
# 但是，在任务的整个生命周期内都不会放弃gpu资源【可能需要重新使用】